In [1]:
import tensorflow as tf

import numpy as np
import os
import time

Read the data

First, look in the text:

In [2]:
# Read, then decode for py2 compat.
text = open('Pride-and-Prejudice.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 723244 characters


In [3]:
# Take a look at the first 250 characters in text
print(text[:500])

Pride and Prejudice

by Jane Austen

Chapter 1

It is a truth universally acknowledged, that a single man in possession of a good fortune, must be 
in want of a wife.


However little known the feelings or views of such a man may be on his first entering a 
neighbourhood, this truth is so well
fixed in the minds of the surrounding families, that he is considered the rightful property of some 
one or other of their daughters.



"My dear Mr. Bennet," said his lady to him one day,


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

78 unique characters


In [5]:
vocab[0:10]

['\n', '\r', ' ', '!', '"', "'", '(', ')', ',', '-']

Process the text

Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
text_as_int

array([40, 68, 59, ..., 17,  1,  0])

The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [7]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

P
r
i
d
e


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Pride and Prejudice\r\n\r\nby Jane Austen\r\n\r\nChapter 1\r\n\r\nIt is a truth universally acknowledged, that a '
'single man in possession of a good fortune, must be \r\nin want of a wife.\r\n\r\n\r\nHowever little known th'
'e feelings or views of such a man may be on his first entering a \r\nneighbourhood, this truth is so we'
'll\r\nfixed in the minds of the surrounding families, that he is considered the rightful property of so'
'me \r\none or other of their daughters.\r\n\r\n\r\n\r\n"My dear Mr. Bennet," said his lady to him one day, "hav'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [9]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [10]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Pride and Prejudice\r\n\r\nby Jane Austen\r\n\r\nChapter 1\r\n\r\nIt is a truth universally acknowledged, that a'
Target data: 'ride and Prejudice\r\n\r\nby Jane Austen\r\n\r\nChapter 1\r\n\r\nIt is a truth universally acknowledged, that a '


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [11]:
# Batch size
#First hyperparameters change increasing Batch size
#larger batch sizes make larger gradient steps than smaller batch sizes for the same number of samples seen.
#The same average Euclidean norm distance from the initial weights of the model, larger batch sizes have larger variance in the distance


BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 20000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(128, 100), dtype=tf.int64, name=None), TensorSpec(shape=(128, 100), dtype=tf.int64, name=None))>

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;

tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)

tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [12]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

#Adding Dorp_rate
drop_rate = 0.1

In [13]:
#2 change in hyper parameter Adding Activitatio layer ='tanh'
#3 Change in hyper parameter Adding kernel initializer glorot uniform to the gru layer
#4 change in hyhper parameter Adding Dropout layer  0.1 drop_rate

#Adding activitaion function to the dense layer

def Model_Building(vocab_size, embedding_dim, rnn_units, batch_size,drop_rate):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        activation ='tanh',
                        return_sequences=True,
                        kernel_initializer='glorot_uniform',
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(drop_rate),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [14]:
#Building the model
model = Model_Building(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE,
    drop_rate=drop_rate)

Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [15]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 78) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 256)          19968     
                                                                 
 gru (GRU)                   (128, None, 1024)         3938304   
                                                                 
 dropout (Dropout)           (128, None, 1024)         0         
                                                                 
 dense (Dense)               (128, None, 78)           79950     
                                                                 
Total params: 4,038,222
Trainable params: 4,038,222
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [17]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
#This gives us, at each timestep, a prediction of the next character index:
sampled_indices

array([33, 73, 52, 54,  1, 69, 52, 47,  2, 10, 29, 63, 42, 32,  6,  4, 34,
       46, 44, 22,  5, 47, 61, 10, 60,  1, 57, 36,  9, 73, 68, 73, 18, 33,
       72, 71, 25, 63, 41, 48,  5, 10,  5, 66, 39, 25, 27, 71, 11, 48, 76,
       60, 18, 42, 42, 15, 25, 52,  7,  4, 66, 76, 58, 22, 77,  5, 17, 57,
       67,  5, 65, 75, 68, 34, 32, 35, 23, 32, 72, 71, 16, 56, 19, 63, 65,
       65, 37, 13,  9,  9, 40, 35, 49,  7, 75, 64, 29,  8, 61, 28])

Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (128, 100, 78)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.3545547


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [19]:
model.compile(optimizer='adam', loss=loss)

Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Execute the training

To keep training time reasonable, use 10 epochs to train the model. 

In [21]:
EPOCHS=15
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/15
55/55 [==============================] - 16s 201ms/step - loss: 3.3780
Epoch 2/15
55/55 [==============================] - 12s 203ms/step - loss: 2.3459
Epoch 3/15
55/55 [==============================] - 12s 203ms/step - loss: 2.1289
Epoch 4/15
55/55 [==============================] - 12s 203ms/step - loss: 1.9404
Epoch 5/15
55/55 [==============================] - 13s 208ms/step - loss: 1.7676
Epoch 6/15
55/55 [==============================] - 14s 209ms/step - loss: 1.6282
Epoch 7/15
55/55 [==============================] - 14s 214ms/step - loss: 1.5123
Epoch 8/15
55/55 [==============================] - 13s 211ms/step - loss: 1.4203
Epoch 9/15
55/55 [==============================] - 13s 209ms/step - loss: 1.3503
Epoch 10/15
55/55 [==============================] - 13s 211ms/step - loss: 1.2972
Epoch 11/15
55/55 [==============================] - 13s 211ms/step - loss: 1.2557
Epoch 12/15
55/55 [==============================] - 13s 213ms/step - loss: 1.2211
Epoch 13/15
5

Generate text

Restore the latest checkpoint
To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [22]:
tf.train.latest_checkpoint(checkpoint_dir)
model = Model_Building(vocab_size, embedding_dim, rnn_units, batch_size=1,drop_rate=drop_rate)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            19968     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dropout_1 (Dropout)         (1, None, 1024)           0         
                                                                 
 dense_1 (Dense)             (1, None, 78)             79950     
                                                                 
Total params: 4,038,222
Trainable params: 4,038,222
Non-trainable params: 0
_________________________________________________________________


The prediction loop

The following code block generates the text:

It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

Get the prediction distribution of the next character using the start string and the RNN state.

Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [23]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [63]:
print(generate_text(model, start_string=u"The Longbourn "))

The Longbourn what I had ever refeeted with a circle 
to a stranger again, Mr. Darcy meet him, when you seem! He mean in with sometimes remarked, she added, "if I am too much expectations and 
general for whisper there to I single me with allation at 
his myself. And general any charge spate of the library. Whth Bingley?" cried he; but his never was ever been worth, "you well as you do the subject; and as they were reing hor's accept him any restion, took the time of prejuicing a sense of the last 
street, and leave THE
the eliest and finding by Lizzy, you une deserving partial consideration of her kindness, wride, and make 
that Pambsorn that mention 
so itserely and complaid us, as a bar night ollube he was atwanded, with some other of the last assoment 
addressed. Jane was ease, before he has



page 
the rest with as her suspected husband many play, and behaved of
the power. 



The mother was duar

of seeing him to say to leave the felt
and news by marriage, and frontinging
